In [98]:
# Import libraries

import warnings
from resources import *
import datetime
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics, losses, optimizers
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from keras import regularizers

warnings.filterwarnings("ignore")

In [14]:
!python resources.py

In [15]:
# Load user items data
recdata = pd.read_csv('recdata.csv', index_col=0)
recdata = recdata.rename(columns={'variable': 'id', 'value': 'owned'})
recdata.head()

,uid,id,owned
0,0,10,1.0
1,1,10,1.0
2,3,10,1.0
3,4,10,1.0
4,10,10,1.0


## Preprocessing

In [16]:
# Use create_interaction_matrix function
interactions = create_interaction_matrix(
    df=recdata, user_col="uid", item_col="id", rating_col="owned"
)


In [17]:
# Create user dictionary using helper function
# user_dict = create_user_dict(interactions=interactions)

# Load games data
gamesdf = pd.read_csv('gamesdata.csv', index_col=0)

# Create game dictionary using helper function
games_dict = create_item_dict(df=gamesdf, id_col='id', name_col='title')

In [18]:
# Load the data
data = interactions.T
# data['y'] = data.apply(lambda row: row.values, axis=1)
data.head()

uid,0,1,2,3,4,5,6,7,8,10,...,29986,29987,29988,29993,29994,29995,29996,29997,29998,29999
id,,,,,,,,,,,,,,,,,,,,,
10,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
20,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
30,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
40,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
50,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train TF model

In [99]:
# Get recs functions

def get_game_title(game_id, game_data):
    title = None
    if game_id in game_data.index:
        title = game_data.loc[game_id, 'title']
    return title

def get_recommendations(model, data, user_id, n, scaler, pca, item_titles):
    # Get the index of the user in the data
    user_index = data.index.get_loc(user_id)

    # Get the user's interactions with the games
    interactions = data.iloc[user_index].values

    # Scale and reduce the interactions using the same scaler and PCA used for training
    interactions_scaled = scaler.transform([interactions])
    interactions_reduced = pca.transform(interactions_scaled)

    # Generate the input features for the model
    input_features = interactions_reduced.reshape(1, -1)

    # Make predictions using the model
    predictions = model.predict(input_features)[0]

    # Get the indices of recommendations
    top_indices = predictions.argsort()[::-1]


    # Print the user's games and the top N recommendations
    print("User's games:")
    user_games = data.iloc[user_index][data.iloc[user_index] != 0].index.tolist()

    for game_id in user_games:
        game_title = item_titles.get(game_id)
        if game_title is not None:
            print("- {}".format(game_title))


    # Get the titles of the top N recommendations
    game_titles = []
    for i in top_indices:
        game_id = data.columns[i]
        game_title = item_titles.get(game_id)
        if game_title is not None and game_id not in user_games:
            game_titles.append(game_title)    

    print("Top recommendations:")
    for i, game_title in enumerate(game_titles):
        print("{}. {}".format(i + 1, game_title))



In [71]:
# Normalize the input data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Apply PCA to reduce the dimensionality of the input data
pca = PCA(n_components=50)
transformed_data = pca.fit_transform(data_scaled)

In [103]:
# Prepare data & create model
train_data, test_data = train_test_split(
    transformed_data, test_size=0.2, random_state=42
)
train_target = train_data.copy()
train_target[train_data != 0] = 0
num_items = transformed_data.shape[1]


# Define the model
model = Sequential(
    [
        Dense(
            64,
            activation="relu",
            input_shape=(transformed_data.shape[1],),
        ),
        Dense(32, activation="relu"),
        Dense(
            16,
            activation="relu",
            kernel_regularizer=regularizers.l2(0.01),
        ),
        Dense(transformed_data.shape[1], activation="softmax"),
    ]
)


In [105]:
# Train model
from keras.metrics import Precision, Recall, AUC

# Log tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


metr = (
    Precision(),
    Recall(),
)
loss = (losses.BinaryCrossentropy(from_logits=True),)
optimizer = optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metr)

# Train the model on the training data
model.fit(
    train_data,
    train_target,
    epochs=400,
    batch_size=16,
    validation_data=(test_data, test_data),
    callbacks=[tensorboard_callback],
)


Epoch 1/400
433/433 [==============================] - 3s 4ms/step - loss: 5.0882e-09 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - val_loss: -0.3945 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 2/400
433/433 [==============================] - 2s 3ms/step - loss: 4.7184e-09 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - val_loss: -0.3795 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 3/400
433/433 [==============================] - 2s 4ms/step - loss: 4.4066e-09 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - val_loss: -0.3907 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 4/400
433/433 [==============================] - 2s 4ms/step - loss: 4.0298e-09 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00 - val_loss: -0.3895 - val_precision_12: 0.0000e+00 - val_recall_12: 0.0000e+00
Epoch 5/400
433/433 [==============================] - 2s 4ms/step - loss: 3.8051e-09 - precision_12: 0.0000e+00 - recall_12: 0.0000e+00

KeyboardInterrupt: 

In [61]:
# Save model
pickle.dump(model, open('model.pkl', 'wb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
...metrics
......auc
.........vars
............0
............1
............2
............3
......mean
.........vars
............0
............1
......precision
.........vars
............0
............1
......recall
.........vars
............0
............1
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-03-14 16:44:10           64
config.json                                    2023-03-14 16:44:10         2955
variables.h5                                   2023-03-14 16:44:10       100312


### Load model

In [62]:
# Load model
pickled_model = pickle.load(open('model.pkl', 'rb'))

Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-03-14 16:44:10           64
config.json                                    2023-03-14 16:44:10         2955
variables.h5                                   2023-03-14 16:44:10       100312
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
...metrics
......auc
.........vars
............0
............1
............2
............3
......mean
.........vars
............0
............1
......precision
.........vars
............0
............1
......recall
.........vars
............0
............1
...vars


### Get recs

In [94]:
user_id = 10
num_recommendations = 5
recommendations = get_recommendations(
    model=model,
    data=data,
    user_id=user_id,
    n=num_recommendations,
    scaler=scaler,
    pca=pca,
    item_titles=games_dict
)


1/1 [==============================] - 0s 29ms/step
User's games:
- Counter-Strike
- Team Fortress Classic
- Half-Life: Source
- Alien Swarm
- Mare Nostrum
- Iron Warriors: T - 72 Tank Command
- Quake IV
- QUAKE
- QUAKE II
- QUAKE II Mission Pack: The Reckoning
- X: Tension
- 688(I) Hunter/Killer
- Xpand Rally
- Bejeweled 2 Deluxe
- Zuma Deluxe
- Bejeweled Deluxe
- Escape Rosecliff Island
- Zuma's Revenge!
- Garry's Mod
- Silverfall
- Company of Heroes - Legacy Edition
- Indiana Jones® and the Fate of Atlantis™
- Shank
- Vegas: Make It Big™
- Bloodline Champions
- Commandos 2: Men of Courage
- Deus Ex: Game of the Year Edition
- Jade Empire™: Special Edition
- Sid Meier's Railroads!
- Railroad Tycoon II Platinum
- Sam &amp; Max 106: Bright Side of the Moon
- Sam &amp; Max 202: Moai Better Blues
- Sam &amp; Max 204: Chariots of the Dogs
- Telltale Texas Hold ‘Em
- RACE 07: Andy Priaulx Crowne Plaza Raceway (Free DLC)
- Civilization IV: Beyond the Sword
- Freedom Force vs. the Third Reic